In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pickle
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap   
import os
mpl.use('Agg')
plt.ioff()

In [4]:
dataset = pd.read_csv('data_earthquake_2000-2017.csv',encoding='latin1', sep=';')
dataset.head()

,Año,Mes,Día,Hora,Minuto,Segundo,Fecha,Latitud,Longitud,Prof.,Magn,Date,Horario,Dtime,time,diff
0,2000,1,2,3,42,12.7,36527,-33.410,-70.880,81.9,2.5,36527,0.154314,36527.15431,1.154314,0.000000
1,2000,1,2,5,3,13.1,36527,-33.787,-71.339,45.8,3.7,36527,0.210568,36527.21057,1.210568,0.056255
2,2000,1,2,18,55,55.0,36527,-31.535,-71.318,35.7,3.8,36527,0.788831,36527.78883,1.788831,0.578263
3,2000,1,2,19,36,25.8,36527,-32.446,-70.923,95.2,2.7,36527,0.816965,36527.81697,1.816965,0.028134
4,2000,1,3,2,41,41.7,36528,-32.986,-71.348,41.8,3.6,36528,0.112288,36528.11229,2.112288,0.295323


In [5]:
%matplotlib inline
TIME_STEP = 9
IMG_SIZE = 32
TIME_WND = 30 #IN DAYS


In [6]:
print(dataset.shape)
dataset = dataset[dataset['Prof.'] >= 60.0 ]
print(dataset.shape)

(85636, 16)
(37195, 16)


In [7]:
zone1 = dataset[dataset['Latitud'] > -23]
zone2 = dataset[(dataset.Latitud < -23) & (dataset.Latitud > -34)] 
zone3 = dataset[dataset['Latitud'] < -34]
print(dataset.shape)
print(zone1.shape)


(37195, 16)
(16389, 16)


In [27]:
def scale(data,new_min, new_max):
    old_max = data['Magn'].max()
    old_min = data['Magn'].min()
    old_range =  old_max - old_min
    new_range = new_max - new_min 
    data['Magn'] = data['Magn'].apply(lambda v: int(((v-old_min)*new_range)/old_range)+new_min)
    return data

zone1 = scale(zone1,25,255)

/home/sergio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
def build_time_series(data,y_col=0):
    FIRST = data.Date[:1].values[0]
    dim_0 = data.shape[0]- TIME_WND
    
    y = []
    series = []
    X = []
    k = 1
    for i in range(dim_0):
        series_df = data[(data.Date >= FIRST+i) & (data.Date <= FIRST+i+TIME_WND)]
        k+=1
        #print('creating x')
        #print(FIRST+i, FIRST+i+TIME_WND)
        series.append(series_df)
        if(k==TIME_STEP):
            #print('creating y')
            #print(FIRST+i+TIME_WND)
            #print(FIRST+i+TIME_WND*2)
            y_df = data[(data.Date > FIRST+i+TIME_WND) & (data.Date < FIRST+i+TIME_WND*2)]
            
            X.append(series)
            y.append(y_df)
            series = []
            k = 1

    return X,y

X,y = build_time_series(zone1)

In [7]:
def init_latlon(data):
    lat_max = data['Latitud'].max()
    lat_min = data['Latitud'].min()
    lon_max = data['Longitud'].max()
    lon_min = data['Longitud'].min()
    lat_m = data['Latitud'].mean()
    lon_m = data['Longitud'].mean()
    
    return lat_max,lat_min,lon_max,lon_min,lat_m,lon_m


lat_max,lat_min,lon_max,lon_min,lat_m,lon_m = init_latlon(zone1)


In [ ]:
def get_marker_color(magnitude):
    # Returns green for small earthquakes, yellow for moderate
    #  earthquakes, and red for significant earthquakes.
    if magnitude < 3.0:
        return ('go')
    elif magnitude < 5.0:
        return ('yo')
    else:
        return ('ro')

def create_pre_images_x(X):
    dirpath = os.getcwd()
    
    for i, serie in enumerate(X):
        series_path = dirpath+'/images/pre_images/series_'+str(i)+'_'
        for j, data in enumerate(serie):
            fig = plt.figure(figsize=(8, 8))
            m = Basemap(llcrnrlon=lon_min,llcrnrlat=lat_min,urcrnrlon=lon_max,urcrnrlat=lat_max,
                        resolution='i',projection='lcc',lat_0=lat_m,lon_0=lon_m)
            m.shadedrelief()
            m.drawcoastlines(color='gray')
            m.drawcountries(color='gray')
            
            if not data.empty:
                
                lat = data['Latitud'].values
                lon = data['Longitud'].values
                magn = data['Magn'].values
                ##print(magn)
                for lon, lat, mag in zip(lon, lat, magn):
                    x,y = m(lon, lat)

                    marker_string = get_marker_color(mag)
                    m.plot(x, y, marker_string, markersize=10)
            

            plt.savefig(series_path+'Xquakemap_'+str(j)+'.jpg')
            plt.close()
    
def create_pre_images_y(y):
    dirpath = os.getcwd()
    for k, data in enumerate(y):
        series_path =  dirpath+'/images/pre_images/series_'+str(k)+'_'
        fig = plt.figure(figsize=(8, 8))
        m = Basemap(llcrnrlon=lon_min,llcrnrlat=lat_min,urcrnrlon=lon_max,urcrnrlat=lat_max,
                    resolution='i',projection='lcc',lat_0=lat_m,lon_0=lon_m)
        m.shadedrelief()
        m.drawcoastlines(color='gray')
        m.drawcountries(color='gray')
        if not data.empty:
            lat = data['Latitud'].values
            lon = data['Longitud'].values
            magn = data['Magn'].values
            for lon, lat, mag in zip(lon, lat, magn):
                x,y = m(lon, lat) 
                marker_string = get_marker_color(mag)
                m.plot(x, y, marker_string, markersize=10)
        plt.savefig(series_path+'Yquakemap.jpg')
        plt.close()


create_pre_images_x(X)
create_pre_images_y(y)

In [12]:
from PIL import ImageChops
import glob
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

dirpath = os.getcwd()
folderName = dirpath+'/images/pre_images'
filePaths = glob.glob(folderName + "/*.jpg")


for filePath in filePaths:
    #print(filePath)
    image=Image.open(filePath)
    im = trim(image)
    im.save(filePath)

In [ ]:
def make_array(dim_x):
    arr = []
    for i in range(dim_x):
        series = []
        for j in range(TIME_STEP-1):
            series.append(np.asarray(Image.open(dirpath+'/images/pre_images/series_{0}_Xquakemap_{1}.jpg'.format(i,j)).convert('RGB')))
        arr.append(np.array(series))
    return arr

def gallery(array, ncols=4):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    # want result.shape = (height*nrows, width*ncols, intensity)
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

make_array(len(X))
for i,series in enumerate(array):
    im = Image.fromarray(gallery(array))
    im.save(dirpath+'/images/gallery/series_{0}_Xquakemap.jpg'.format(i))

3429